# Stock price collector

In [1]:
from pandas_datareader import data
from pandas_datareader._utils import RemoteDataError
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import csv
import re
from datetime import datetime

import os.path
from os import path
import time

# import sklearn
# import scipy

# from sklearn.naive_bayes import GaussianNB
# import lightgbm as lightgbm
# from xgboost import XGBClassifier
# import keras

# Get Started
# https://github.com/Refinitiv-API-Samples/Article.EikonDataAPI.DotNet.Library
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/quick-start
# API limits
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/documentation?content=49692&type=documentation_item
import eikon as ek 

from pylab import rcParams
plt.rcParams['figure.figsize'] = 16, 8

In [2]:
fontsize_reg = 12
fontsize_title = 16
from pylab import rcParams
rcParams['figure.facecolor'] = '1'
rcParams['figure.figsize'] = [8.0, 3.5]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 600

rcParams['font.size'] = 12
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'large'

In [3]:
# Зададим path для папки проекта и для папок файлов
path_project_folder = 'C:/DAN/t_systems/trade_project/'
folder_name_headlines = 'backup_headlines/'
folder_name_logger = 'logger/'
folder_name_price_timeseries = 'price_timeseries/'

In [48]:
# Для работы с API нужен запущенный терминал Thomson Reuters Eikon
# Зададим API ключ - не даёт доступ сам по себе
ek.set_app_key('df6c94f7d0fb4b4e80533d1c2f86daf402ce52c0') 
# ccebd1ff788d4574818be93e35c4741925ac214e
# df6c94f7d0fb4b4e80533d1c2f86daf402ce52c0

In [66]:
df = ek.get_timeseries(
    ["MSFT.O"],
    start_date="2010-01-01",
    end_date="2022-01-30",
    fields=['TIMESTAMP', 'VALUE', 'VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
)

df

MSFT.O,VALUE,VOLUME,HIGH,LOW,OPEN,CLOSE,COUNT
Date,,,,,,,
2010-01-04,<NA>,38414185,31.1000,30.59,30.620,30.950,-1
2010-01-05,<NA>,49758862,31.1000,30.64,30.850,30.960,-1
2010-01-06,<NA>,58182332,31.0800,30.52,30.880,30.770,-1
2010-01-07,<NA>,50564285,30.7000,30.19,30.630,30.452,-1
2010-01-08,<NA>,51201289,30.8800,30.24,30.280,30.660,-1
...,...,...,...,...,...,...,...
2021-02-25,<NA>,39542235,234.5852,227.88,232.080,228.990,506081
2021-02-26,<NA>,37819227,235.3700,229.54,231.525,232.380,442934
2021-03-01,<NA>,25332837,237.4700,233.15,235.900,236.940,295565


In [67]:
df is None

False

In [11]:
def get_name_of_latest_file_in_folder(folder_name):
    """
    Get all file names in selected direction.
    Return the latest file in that directory.
    """
    files_creation_dates = pd.DataFrame(data = [], columns=[
        "folder_name", "file_name", "num_time", "str_time"
    ])

    files_creation_dates['file_name'] = os.listdir(folder_name)
    files_creation_dates['folder_name'] = folder_name
    files_creation_dates['file_folder_and_name'] = files_creation_dates['folder_name'] + files_creation_dates['file_name']
    files_creation_dates
    for i in range(0, len(files_creation_dates)):
        file_folder_and_name = files_creation_dates['file_folder_and_name'][i]
        files_creation_dates['num_time'][i] = os.path.getctime(file_folder_and_name)
        files_creation_dates['str_time'][i] = time.ctime(os.path.getctime(file_folder_and_name))

    latest_file_dir = files_creation_dates.sort_values(by=['num_time'], ascending=False)['file_folder_and_name'].iloc[0]
    
    return latest_file_dir

In [5]:
# Read the file with all news texts to create a list with rics in it
latest_file = get_name_of_latest_file_in_folder(path_project_folder + folder_name_headlines)
all_headlines_df = pd.read_csv(latest_file, sep=';')
all_headlines_df['versionCreated'] = pd.to_datetime(all_headlines_df['versionCreated'])
print(all_headlines_df.shape, latest_file)

(375688, 6) C:/DAN/t_systems/trade_project/backup_headlines/all_headlines_df_2021-02-24_v1.csv


In [50]:
# Create the file with rics from all news texts dataset
# rics_list = list(all_headlines_df['ric'].value_counts().index)
# print(len(rics_list), rics_list)

rics_list = ['JPM.N', 'AMZN.OQ', 'FB.OQ', 'AAPL.OQ', 'MSFT.OQ', 'GOOGL.OQ', 'TSLA.OQ', 'BABA.N', 'PFE.N', 'RWEG.DE', 'VOWG_p.DE', 'BAC.N', 'NFLX.OQ', 'WMT.N', 'SIEGn.DE', 'LHAG.DE', 'DBKGn.DE', 'DIS.N', 'JNJ.N', 'INTC.OQ', 'DAIGn.DE', 'T.N', 'BMWG.DE', 'MRK.N', 'BAYGn.DE', 'VZ.N', 'SAPG.DE', 'EONGn.DE', 'KO.N', 'CBKG.DE', 'MA.N', 'ALVG.DE', 'NKE.N', 'TKAG.DE', 'DTEGn.DE', 'NVDA.OQ', 'V.N', 'CMCSA.OQ', 'BRKa.N', 'UNH.N', 'BASFn.DE', 'PG.N', 'HD.N', 'TMUS.OQ', 'PYPL.OQ', 'ADSGn.DE', 'DPWGn.DE', 'CONG.DE', 'LUMN.N', 'PSMGn.DE', 'MRCG.DE', 'IFXGn.DE', 'FREG.DE', 'HNKG_p.DE', 'FMEG.DE', 'ATUS.N', 'ADBE.OQ', 'CHTR.OQ', 'VNAn.DE', 'JCOM.OQ', 'HEIG.DE', 'O2Dn.DE', 'PDD.OQ', 'UTDI.DE', 'VG.OQ', 'FNTGn.DE', 'BEIG.DE', 'DRIG.DE', 'TIGO.OQ', 'BAND.OQ', 'TC1n.DE', 'IRDM.OQ', 'CABO.N', 'CCOI.OQ', 'USM.N', 'NFN.DE', 'E4CG.DE', 'PH6.F', 'IVV.P', 'VTI.P']

In [103]:
import time

def get_timeseries_of_rics_to_folder(
    list_of_rics,
    folder_for_timeseries_file=path_project_folder+folder_name_price_timeseries,
    timeseries_interval=str('daily'),
    timeseries_start_date=str("2010-01-01"),
    timeseries_end_date=str("2021-06-01"),
    timeseries_is_adjusted=True,
    sleep_between_rics=0):
    
    for ric_now in list_of_rics:
        index_of_ric_now = list_of_rics.index(ric_now)
        
        print(index_of_ric_now, ric_now, end=', ')
        
        # Try to get the data via API untill the number of attempts is over
        timeseries_df = None
        number_of_attempts=1
        while (timeseries_df is None) and (number_of_attempts < 3):
            try:
                # Get df via Eikon Data API
                # Possible values of interval: 
                # 'tick', 'minute', 'hour', 
                # 'daily', 'weekly', 'monthly', 'quarterly', 'yearly' (Default 'daily')
                corax_param = 'adjusted' if timeseries_is_adjusted else 'unadjusted'
                timeseries_df = ek.get_timeseries(
                    [ric_now],
                    interval=timeseries_interval,
                    start_date=timeseries_start_date,
                    end_date=timeseries_end_date,
                    fields=['TIMESTAMP', 'VALUE', 'VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
                    corax=corax_param
                )
            except Exception:
                print('ERROR, Exception. Request fails or if server returns an error.')
                time.sleep(5)
                pass
            except ValueError:
                print('ERROR, ValueError. A parameter type or value is wrong.')
                time.sleep(5)
                pass
            
            print('Attempt', number_of_attempts, end=', ')
            
            if timeseries_df is None: # ERROR
                print('error with that attempt', end=', ')
                time.sleep(60)
            number_of_attempts = number_of_attempts + 1
            
        # End of while
        
        # If it fail several attempts then print the final error message
        if timeseries_df.columns.name == None:
            print(
                'ERROR WITH THIS RIC:', 
                ric_now, 
                timeseries_interval, 
                timeseries_start_date, 
                timeseries_end_date,
                corax_param
            )
            pass
        else:
            print('NOT EMPTY', end=', ')
        
        # Add ric and date columns
        timeseries_df['ric'] = timeseries_df.columns.name 
        # Check index type to do not reset_index again
        if type(timeseries_df.index[0]) != int:
            timeseries_df['Date'] = timeseries_df.index
            timeseries_df = timeseries_df.reset_index(drop=True, inplace=False)

        # Save created df with one ric. File name contains ric name
        file_name = 'price_timeseries' + '_' + timeseries_interval + '_' + corax_param + '_' + timeseries_df.columns.name
        file_path =  folder_for_timeseries_file + file_name + '.csv'
        timeseries_df.to_csv(file_path, sep = ';', index=False)
        print()
        print(' '*(len(ric_now) + len(str(index_of_ric_now))), ', ', end='')
        print(file_path, end=', ')
        
        print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        
        if sleep_between_rics > 0:
            time.sleep(sleep_between_rics)

3333


In [85]:
rics_list[35]

'NVDA.OQ'

In [ ]:
get_timeseries_of_rics_to_folder(
    list_of_rics=rics_list,
    folder_for_timeseries_file=path_project_folder+folder_name_price_timeseries,
    timeseries_interval='daily',
    timeseries_start_date="2010-01-01",
    timeseries_end_date="2021-06-01",
    sleep_between_rics=20)

0 JPM.N, Attempt 1, df is not empty, 
       , C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_daily_adjusted_JPM.N.csv, 03/03/2021 23:28:45


In [194]:
list_of_files_to_merge = os.listdir(path_project_folder + folder_name_price_timeseries)
print(list_of_files_to_merge)

['price_timeseries_Daily_AAPL.OQ.csv', 'price_timeseries_Daily_AMZN.OQ.csv', 'price_timeseries_Daily_BABA.N.csv', 'price_timeseries_Daily_BAC.N.csv', 'price_timeseries_Daily_FB.OQ.csv', 'price_timeseries_Daily_GOOGL.OQ.csv', 'price_timeseries_Daily_JPM.N.csv', 'price_timeseries_Daily_MSFT.OQ.csv', 'price_timeseries_Daily_PFE.N.csv', 'price_timeseries_Daily_RWEG.DE.csv', 'price_timeseries_Daily_TSLA.OQ.csv', 'price_timeseries_Daily_VOWG_p.DE.csv']


In [206]:
all_stock_prices = pd.DataFrame(
    data = [], 
    columns=[])

for file_now in list_of_files_to_merge:
    file_path = path_project_folder + folder_name_price_timeseries + file_now
    timeseries_df = pd.read_csv(file_path, sep=';')
    print(file_path, timeseries_df.shape)
    
    all_stock_prices = pd.concat([all_stock_prices, timeseries_df], ignore_index=True)
    time.sleep(0)

C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_AAPL.OQ.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_AMZN.OQ.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_BABA.N.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_BAC.N.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_FB.OQ.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_GOOGL.OQ.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_JPM.N.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_MSFT.OQ.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_PFE.N.csv (39, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_RWEG.DE.csv (42, 9)
C:/DAN/t_systems/trade_project/price_timeseries/price_timeseries_Daily_TSLA.OQ.csv (39, 9)
C:/DAN/

In [210]:
all_stock_prices

,VALUE,VOLUME,HIGH,LOW,OPEN,CLOSE,COUNT,ric,Date
0,NaN,35418103,133.600,126.76,133.56,129.41,242479,AAPL.OQ,2021-01-04
1,NaN,19952821,131.730,128.44,128.98,131.01,124892,AAPL.OQ,2021-01-05
2,NaN,34918981,131.040,126.39,127.53,126.60,252826,AAPL.OQ,2021-01-06
3,NaN,23977200,131.625,127.87,128.38,130.92,132419,AAPL.OQ,2021-01-07
4,NaN,22460593,132.630,130.23,132.50,132.05,162461,AAPL.OQ,2021-01-08
...,...,...,...,...,...,...,...,...,...
469,NaN,825524,173.720,170.64,171.30,172.62,16174,VOWG_p.DE,2021-02-24
470,NaN,799527,174.300,171.08,174.12,171.08,12579,VOWG_p.DE,2021-02-25
471,NaN,1559222,175.480,167.00,169.00,173.02,23009,VOWG_p.DE,2021-02-26
472,NaN,813177,176.700,173.40,176.00,175.82,13983,VOWG_p.DE,2021-03-01
